---

# Interpretable Machine Learning: Shapley Values
#### United Lunch & Learn: June 6, 2019

_Author: Carleton Smith_

---

## Tutorial Outline

- Install/import packages
- Acquire data
- Initial EDA
- Prepare dataset
- Overview of Shapley Values
    - One
    - Two
    - Three

---
## Install Packages

In [ ]:
import sys
!conda install -yc conda-forge --prefix {sys.prefix} shap

---
## Import Packages

In [ ]:
import os
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 12
plt.style.use("fivethirtyeight")

## Brief Introduction

Before demonstrating how to use Shapley Values for machine learning, let's discuss what they are in the first place. This explanation is based on these two papers:

1. [A Unified Approach to Interpreting Model Predictions](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions.pdf)
2. [Consistent Individualized Feature Attribution for Tree Ensembles](https://arxiv.org/pdf/1802.03888.pdf)

#### TL;DR

Shapley Values originated in game theory and are named after famed mathematician and Nobel Prize winner, [Lloyd Shapley](https://en.wikipedia.org/wiki/Lloyd_Shapley). The purpose of Shapley Values from a game theory perspective is to solve the problem of assigning appropriate credit to individual players in a cooperative game. In recent years, researchers have adopted Shapley Values to assign "credit" to features for predictions produced by a complex model.


---
## Why do we care about interpretable machine learning?

There are many reasons.

- Establish trust in the model
- Better understand underlying processes
- Provide insights in how to improve the model
- Ethical and fairness risks

In addition to Shapley Values, several methods exist on the market for this purpose:

- LIME
- DeepLIFT
- Layer-Wise Relevance Propagation

What sets Shapley Values apart (according to the [the first paper listed above](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions.pdf)) is that Shapely values are the only one of these method that satisfy all three of these feature importance quality requirements:

1. **Local Accuracy** - a simple model explaining a complex one around a particular point should produce the same output given the same inputs
2. **Missingness** - if a feature is missing in the input space, it should not appear in the feature attribution
3. **Consistency** - if a feature is increasing in it's contribution to the outcome, then that feature should increase in it's importance

What ends up happening with many of the other methods is that you can come up with counter-examples where you cannot acheive all 3 of these properties at the same time. Shapley Values are the only solution among these that do.

### Game Theory Context Example: A Soccer Team

Original problem proposed in game theory was that sometimes you have situations where a group of people are working together to acheive an outcome. A good example of this is a soccer team.

_Soccer Team_:
- 11 players
- Well forumated positions

How important are each one of those 11 players to the overall performance of the team? There's just one outcome, but 11 people contributing.

Imagine you are running an experiment with a whole stadium of soccer players and you are creating differet coalitons of soccer players and composing teams of them. Sometimes you have teams of 1 person.

Let's assume the scores are more continuous - 1 and 100 range.

1 player might be really good, but by themselves, not a good team. Perhaps by themself score a 10.
Then you add a player, and that person is also really good, so they are a 9 by themself. But instead of linearly adding them together, the synergistic affect would be a 21 or 22. Then maybe you have a 3rd person, who is a 5, but plays well with the first two, so they add in +8. Then you add in a fourth that is a 7 by themselves, but they get into fights with the first two, so they only add +2 when you add them in. Then you add a 5th one. And this one is a goalie, so this is a position that is totally different than any others, so that might be very valuable. Then you add a 6th one, and they're also a goalie, so that's not as valuable. If they were the first goalie, then it would be valuable, but the order of being added matters. You're trying to understand the coalition that you're creating here, but it dependes on the diff combinations of people and the order in which they enter the game. Since this depends on the unique cominations of individuals and the order in which they are added, the credit attribution problem becomes faily complex.

The question is, is there some way we can summarize value? At the end of the day, there may just be people that are consistently higher impact or lower impact if you were to add up all the possible combinations coalitions.

This is a very large multiplication For each one of the scenarios that you construct, you pay attention to what changes in the overall credit that the team gets when you add in a particular actor to this coalition. Add in all of those contributions up by each player, then divide by number of scenarios you have and that's the Shapley Value for that associated with that person. Fairly straightforward process to calculate, but is very computationally expensive.

When you have potentially large compositions of coalitions, then you have to go through the exercise of knowing what each of those credit assignments are.

Tie back to ML: Instead of the soccer score, we care about a model's prediction of a particular case. Instead of soccer players, we are concerned with what features are playing the biggest role in making this prediction. A Shapley Value tells you how important each feature is for a particular prediction.



---
## Acquire Data

In [ ]:
adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                    na_values=' ?',
                    header=None)

If the above line hangs, then uncomment the line below

In [ ]:
# adult = pd.read_csv('./datasets/adult.data.txt', header=None, na_values=' ?')

---
## Quick Preprocessing/EDA

- Add column headers
- Understand dataset
    - how many rows/columns?
    - what does a row represent?
    - what is our target variable?
- Check for missing values
- Check data types
- Check for unbalanced target variable

In [ ]:
adult.head()

### Add Column Headers

**FEATURES**

1. `age`: continuous.
2. `workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
3. `fnlwgt`: continuous.
4. `education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
5. `education-num`: continuous.
6. `marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
7. `occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
8. `race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
9. `sex`: Female, Male.
10. `capital-gain`: continuous.
11. `capital-loss`: continuousm
12. `hours-per-week`: continuous.
13. `native-country`: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [ ]:
features = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country',
    'income',
]

In [ ]:
# assign column names
adult.columns = features
adult.head()

In [ ]:
# how many rows a columns?
adult.shape

In [ ]:
# any missing values?
adult.isnull().sum()

In [ ]:
# what are the data types?
adult.dtypes

In [ ]:
# what is the distribution of our target variable?
adult['income'].value_counts()

---
## Preprocessing

In the interest of time, I packaged these preprocessing steps into `Pipelines`.

**PREPROCESSING STEPS**
1. Separate target variable from features - sklearn requires this.
2. Peform a train-test split - Always do this before manipulating dataset
3. With training data:
    - **SEPARATE** numeric columns from categorical ones
    - **NUMERIC DF** preprocessing:
        - Replace nan values
        - Standardize features
   
    - **CATEGORICAL DF** preprocessing:
        - Replace nan values
        - Create dummy variables
    - **CONCATENATE** numeric and categorical DF
    - **ENCODE** target variable
<br>
<br>
4. Package these steps into a `Pipeline`

In [ ]:
# make a list of numeric and categorical column names
num_cols = [col for col in adult.columns if adult[col].dtype != 'object']
cat_cols = [col for col in adult.columns if col not in num_cols + ['income']]

# separate features from target variable
X = adult.drop('income', axis=1)
y = adult['income']

# perform a train-test split.... why? stratify on y?
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=24
)

def feature_extractor(df):
    return df.drop('income', axis=1)


def categorical_extractor(df):
    return df.select_dtypes(include=['object'])


def numeric_extractor(df):
    return df.select_dtypes(exclude=['object'])

# create custom transformers
cat_transformer = FunctionTransformer(categorical_extractor, validate=False)
num_transformer = FunctionTransformer(numeric_extractor, validate=False)

# make numeric pipe
num_pipe = Pipeline([
    ('numeric_transformer', num_transformer),
    ('num_im', SimpleImputer(strategy='median')),
    ('StandardScaler', StandardScaler())
])

# make categorical pipe
cat_pipe = Pipeline([
    ('cat_transformer', cat_transformer),
    ('cat_im', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder())
])


# make FeatureUnion
feat_union = FeatureUnion([
    ('num_pipe', num_pipe),
    ('cat_pipe', cat_pipe)
])

# make final feature pipe
feature_pipe = Pipeline([
    ('feat_union', feat_union)
])

#### Use this pipeline to _fit_ and _transform_ `X_train`

In [ ]:
# fit and transform training data
X_train_prepared = pd.DataFrame(
    feature_pipe.fit(X_train).transform(X_train),
    index=X_train.index,
    columns=X_train.columns)
X_train_prepared.head()

In [ ]:
# transform testing data
X_test_prepared = pd.DataFrame(
    feature_pipe.transform(X_test),
    index=X_test.index,
    columns=X_test.columns)
X_test_prepared.head()

#### Use `LabelEncoder` to transform the `income` to be numeric

In [ ]:
y_train[:5]

In [ ]:
# fit and transform y_train
le = LabelEncoder()
y_train_encoded = pd.Series(le.fit_transform(y_train), index=y_train.index)
y_train_encoded[:5]

In [ ]:
# transform y_test
y_test_encoded = pd.Series(le.transform(y_test), index=y_test.index)
y_test_encoded[:5]

#### Calculate Baseline

In [ ]:
y_test_encoded.value_counts()[0] / y_test_encoded.value_counts().sum()

## Create an Explainable Model: `GradientBoostingClassifier`

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
rf = RandomForestClassifier(
    n_estimators=10,
    class_weight='balanced',
    oob_score=True
)
rf.fit(X_train_prepared, y_train_encoded)

In [ ]:
# make predictions for training and test:
y_pred_train = rf.predict(X_train_prepared)
y_pred_test = rf.predict(X_test_prepared)

In [ ]:
print('CLASSIFICATION METRICS FOR TRAINING: \n')
print(classification_report(y_train_encoded, y_pred_train))
print('#########################################################\n')

print('CLASSIFICATION METRICS FOR TESTING: \n')
print(classification_report(y_test_encoded, y_pred_test))

In [ ]:
accuracy_score(y_test_encoded, y_pred_test)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(rf, X_train_prepared, y_train_encoded, scoring='recall', cv=5)

#### Print Top 10 Features

In [ ]:
feat_imp_lst = list(zip(X_train_prepared.columns, rf.feature_importances_))
feat_lst = sorted(feat_imp_lst, key=lambda x: x[1], reverse=True)
for tup in feat_lst[:10]:
    print(tup)

## Shapley values

The `shap` package includes a C++ optimized implementation for several popular Python tree models.

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_train_prepared)

Scratch Work

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_lst = {
    "n_estimators": np.arange(10, 105, 10),
    "max_depth": [None, 30, 10, 5],
    "class_weight": ['balanced'],
    "oob_score": [True],
}

In [ ]:
rs = RandomizedSearchCV(rf, param_distributions=param_lst, verbose=2)

In [ ]:
rs.fit(X_train_prepared, y_train_encoded)

In [ ]:
rs.best_estimator_